Python version 3.10; Packages: ldpc, bposd, networkx

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg
from scipy.sparse import csc_matrix, csr_matrix, lil_matrix, identity, diags
import scipy.sparse
from scipy.io import mmwrite
from ldpc.codes import rep_code, hamming_code
from bposd.hgp import hgp
from bposd.css import css_code
from ldpc.code_util import construct_generator_matrix, compute_code_parameters, estimate_code_distance, compute_exact_code_distance
from ldpc.mod2 import rank
import networkx as nx
from networkx.algorithms.bipartite import configuration_model, biadjacency_matrix
from matplotlib import rcParams
rcParams['font.size'] = 14
rcParams['text.usetex'] = True

# $K_{3,3}$ cycle code

In [4]:
checks = [[1,2,3],[4,5,6],[7,8,9],[1,4,7],[2,5,8],[3,6,9]]
H = np.zeros([6,9], dtype=int)
for i in range(6):
    H[i,np.array(checks[i])-1] = [1,1,1]
print(H.sum(axis=0), H.sum(axis=1))

[2 2 2 2 2 2 2 2 2] [3 3 3 3 3 3]


In [5]:
H = H[:-1]
print(H)

[[1 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 1]
 [1 0 0 1 0 0 1 0 0]
 [0 1 0 0 1 0 0 1 0]]


# Random LDPC (config model)

In [2]:
n = 12
d_v = 3
d_c = 4

for i in range(1000):
    graph = configuration_model(d_v*np.ones(n,dtype=int), d_c*np.ones(int(n*d_v/d_c),dtype=int), create_using=nx.Graph())
    H = biadjacency_matrix(graph, row_order=np.arange(n)).T.toarray()
    # print(estimate_code_distance(H, timeout_seconds=0.5)[0])
    d = compute_exact_code_distance(H)
    if d >= 6: break

In [3]:
print(compute_code_parameters(H))
print(H.sum(0), H.sum(1))

(12, 3, 6)
[2 3 3 3 3 3 3 2 3 3 2 3] [4 4 4 4 3 4 3 3 4]


# Adjacency graph vertex coloring

In [4]:
def get_check_coloring(H):
    # H^T.H for bits, H.H^T for checks
    A = (H@H.T != 0).astype(int)
    # print(A)
    G = nx.from_numpy_array(A, create_using=nx.Graph())
    color_dict = nx.greedy_color(G, strategy='independent_set')
    num_colors = max(list(color_dict.values())) + 1
    coloring = []
    for i in range(num_colors):
        coloring.append([key for key, value in color_dict.items() if value == i])
    return coloring

# Sample random LDPC codes

In [6]:
n = 12
d_v = 3
d_c = 4
min_dist = 6

for i in range(10000):
    graph = configuration_model(d_v*np.ones(n,dtype=int), d_c*np.ones(int(n*d_v/d_c),dtype=int), create_using=nx.Graph())
    H = biadjacency_matrix(graph, row_order=np.arange(n)).T.toarray()
    d = compute_exact_code_distance(H)
    if d >= min_dist:
        coloring = get_check_coloring(H)
        if len(coloring)<=4:
            print('Success!')
            break

Success!


In [409]:
print([len(color) for color in coloring])

[3, 3, 2, 1]


In [410]:
print(compute_code_parameters(H))
print(H.sum(0), H.sum(1))

(12, 3, 6)
[3 2 3 2 2 3 2 2 3 3 3 2] [3 4 2 4 4 3 3 3 4]


# Sample irregular LDPC codes

In [437]:
n = 10
m = 7
n_c = 3      # Fixed
n_v = 3      # Maximum (variable)
min_dist = 5    # Adjust minimum distance as needed
num_samples = 5000

for i in range(num_samples):
    H = np.zeros([m,n], dtype=int)
    for i in range(m):
        deg_v = np.sum(H, axis=0)
        vertices = np.where(deg_v < n_v)[0]
        ones = np.random.choice(vertices,n_c,replace=True)
        for j in range(n_c):
            H[i][ones[j]] = 1
    
    k = n - rank(H)
    if k == n-m:                    # Only look for full-rank H
        d = compute_exact_code_distance(H)
        if d >= min_dist:
            coloring = get_check_coloring(H):
            if len(coloring)<=4:
                print('n = {}, k = {}, d = {}'.format(n,k,d))
                break

n = 10, k = 3, d = 5


In [450]:
print(compute_code_parameters(H))
coloring = get_check_coloring(H)
print([len(color) for color in coloring])
print(H.sum(0), H.sum(1))

(9, 4, 4)
[3, 2]
[2 2 1 2 2 1 2 2 1] [3 3 3 3 3]


# Analyze code parameters

In [462]:
HGPcode = hgp(h1=H, h2=H, compute_distance=True)
HGPcode.name = 'HGP code'
HGPcode.test()

HGP code
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -HGP code is a valid CSS code w/ params [106,16,4]


True

In [475]:
print(HGPcode.hx.sum())
print(HGPcode.hx.sum()/HGPcode.hx.shape[1])
print(HGPcode.hx.shape[1])

210
1.9811320754716981
106
